<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/03%20Machine%20Learning/notebooks/09-Regresion-Logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión Logística

En esta notebook usaremos la implementación de scikit-learn de la regresión logística [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Primero veremos algunos ejemplos didacticos para entender el funcionamiento del clasificador. Después, resolveremos ejemplos usando algunos datasets reales.

Recuerda la simbología de las secciones:

* 🔽 Esta sección no forma parte del proceso usual de Machine Learning. Es una exploración didáctica de algún aspecto del funcionamiento del algoritmo.
* ⚡ Esta sección incluye técnicas más avanzadas destinadas a optimizar o profundizar en el uso de los algoritmos.
* ⭕ Esta sección contiene un ejercicio o práctica a realizar. Aún si no se establece una fecha de entrega, es muy recomendable realizarla para practicar conceptos clave de cada tema.

In [ ]:
#@title Función para graficar la frontera de decisión

from sklearn.inspection import DecisionBoundaryDisplay
import numpy as np

def graficar_FD(X,y,clf,h=0):
    '''
    X es todas las instancias las cuales incluiremos en el gráfico
    '''
    assert X.shape[1] == 2   # Sólo funciona para datos en dimensión 2
    feature_1, feature_2 = np.meshgrid(
    np.linspace(X[:,0].min()-h, X[:, 0].max()+h),
    np.linspace(X[:, 1].min()-h, X[:, 1].max()+h)
    )
    grid = np.vstack([feature_1.ravel(), feature_2.ravel()]).T
    y_grid_pred = clf.predict(grid)
    y_grid_pred = y_grid_pred.reshape(feature_1.shape)
    display = DecisionBoundaryDisplay(
    xx0=feature_1, xx1=feature_2, response=y_grid_pred
    )
    display.plot()
    display.ax_.scatter(
        X[:, 0], X[:, 1], c=y, edgecolor="black"
    )
    plt.show()
    return display.ax_

## Ejemplo 1(a)

En el siguiente ejemplo vemos cómo generar fronteras de decisión más complejas.

In [ ]:
import pandas as pd

url = 'https://github.com/DCDPUAEM/DCDP/raw/main/03%20Machine%20Learning/data/binary-classification-data.csv'
df = pd.read_csv(url,header=None)
df

Observemos los datos

In [ ]:
import matplotlib.pyplot as plt

X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

plt.figure()
plt.scatter(X[:,0],X[:,1],c=y,s=60)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7)

Hacemos una regresión logística con los parámetros por default y vemos su accuracy en el conjunto de entrenamiento y prueba.

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train,y_train)

print(f"Accuracy en el entrenamiento: {clf.score(X_train,y_train)}")
print(f"Accuracy en la prueba: {clf.score(X_test,y_test)}")

Hacemos una regresión logística con polinomial features y vemos su accuracy en el conjunto de entrenamiento. La justificación de usar la regresión logística lineal, junto con polinomial features, para adaptar el módelo al caso polinomial es similar a la justificación en el caso de regresión lineal.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

p_feats = PolynomialFeatures(2,include_bias=False)
log_reg = LogisticRegression(penalty='l2', C=1, solver='newton-cholesky')

pl = Pipeline([('pf',p_feats),
               ('clf',log_reg)])
pl.fit(X,y)
pl.score(X,y)

In [ ]:
_ = graficar_FD(X_test,y_test,pl,h=0.5)

## Ejemplo 1(b)

En este ejemplo veremos el efecto de la regularización como herramienta para prevenir el *overfitting*. Veremos un dataset con muchas features y varias de ellas correlacionadas.

Además, usaremos la regularización L1 para eliminar features.

In [ ]:
from sklearn.datasets import make_classification

# X, y = make_classification(n_samples=300,
#                            n_features=60,
#                            n_informative=20,
#                            n_repeated=3,
#                            n_redundant=20,
#                            n_classes=2,
#                            n_clusters_per_class=3,
#                            class_sep = 0.5,
#                            random_state=49)

X, y = make_classification(n_samples=500,
                           n_features=100,
                           n_informative=20,
                           n_repeated=3,
                           n_redundant=20,
                           n_classes=2,
                           class_sep = 0.5,
                           random_state=57)

### Entrenamiento y evaluación

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75,random_state=1001)

Realicemos el entrenamiento. El rendimiento parece ser bueno en el conjunto de entrenamiento pero en el conjunto de prueba es malo. **Esta es una señal de overfitting**.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty=None)
lr.fit(X_train,y_train)
print(f"Training score: {lr.score(X_train,y_train)}")
print(f"Test score: {lr.score(X_test,y_test)}")

### Regularización

Usamos un clasificador con regularización

In [ ]:
lr2 = LogisticRegression(C=0.1,penalty='l1',solver='liblinear')
lr2.fit(X_train,y_train)
print(f"Training score: {lr2.score(X_train,y_train)}")
print(f"Test score: {lr2.score(X_test,y_test)}")

🔽 Observemosla distribución de las magnitudes de los coeficientes

In [ ]:
import seaborn as sns

normas = np.array([np.linalg.norm(x) for x in lr2.coef_[0]])

plt.figure()
sns.histplot(normas)
plt.show()

Podemos ver que hay varios coeficientes que son exactamente cero. Observar que el número es parecido al número de features redundantes + número de features repetidas

In [ ]:
coefs = lr2.coef_
num_zeros = coefs[coefs==0].shape[0]
print(f"Número de coeficientes = 0: {num_zeros}")

### ⚡ ROC-AUC

Usaremos el area bajo la curva ROC como métrica de rendimiento del clasificador. Para esto necesitamos las probabilidades de las predicciones.

Hay varias maneras de calcularlo:

* Si el clasificador tiene un método `predict_proba`, usamos [roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html).
* Si no, usamos [RocCurveDisplay.from_estimator](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.RocCurveDisplay.html#sklearn.metrics.RocCurveDisplay.from_estimator)

Los siguientes estimadores tienen el método `predict_proba`: `SVC`, `DecisionTreeClassifier`, `RandomForestClassifier`

In [ ]:
y_pred_probs = lr2.predict_proba(X_test)

y_pred_probs.shape

In [ ]:
y_pred = lr2.predict(X_test)

print(y_pred.shape)

print(y_pred_probs[:2])
print(y_pred[:2])

Al ser clasificación binaria, observa que sólo necesitamos la primer columna de la matriz `y_pred_probs`

In [ ]:
from sklearn.metrics import roc_auc_score

score = roc_auc_score(y_test, y_pred_probs[:,1])
print(f"ROC-AUC score: {score}")

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs[:,1])

plt.figure()
plt.suptitle("Curva ROC")
plt.plot(fpr,tpr,color='red')
plt.plot([0,1],[0,1],linestyle='--',color='gray')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.show()

## Ejemplo 3: MNIST

Usaremos el dataset de dígitos escritos a mano. Tenemos dos versiones:

* Usando `keras`, es el dataset MNIST completo. Son 70,000 imágenes de $28\times 28$, divididas en 60,000 de entrenamiento y 10,000 de prueba.
* Usando `sklearn`, es una versión reducida. Son 1797 imágenes de $8\times 8$.

Por practicidad, usaremos la segunda opción.

### Versión Keras

In [ ]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1,28*28)
X_test = X_test.reshape(-1,28*28)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train[28]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(7,2))
for idx, (image, label) in enumerate(zip(X_train[:5], y_train[:5])):
    plt.subplot(1, 5, idx + 1)
    plt.imshow(np.reshape(image, (28,28)), cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title(label)

### Versión Scikit-learn

Obtenemos el dataset

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
y = digits.target

print(X.shape)

Veamos cómo se ve una instancia del conjunto de datos

In [ ]:
X[28]

Mostramos algunas instancias de entrenamiento.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,2))
for idx, (image, label) in enumerate(zip(X[:5], y[:5])):
    plt.subplot(1, 5, idx + 1)
    plt.imshow(np.reshape(image, (8,8)), cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title(label)

Hacemos la división en entrenamiento y prueba.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=128)
X_train.shape, X_test.shape

## Entrenamiento y evaluación

Realizamos el entrenamiento y realizamos las predicciones sobre el conjunto de prueba. Reporta las métricas de rendimiento (accuracy, recall, precision) y la mátriz de confusión con el conjunto de prueba.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
import seaborn as sns

clf = LogisticRegression(solver='liblinear')
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print(f"Accuracy: {round(accuracy_score(y_test,y_pred),3)}")
print(f"Recall: {round(recall_score(y_test,y_pred,average='macro'),3)}")
print(f"Precision: {round(precision_score(y_test,y_pred,average='macro'),3)}")

plt.figure(figsize=(3,3))
cm = confusion_matrix(y_test,y_pred)
s_cm = sns.heatmap(cm,cmap='plasma',annot=True, fmt='g')
plt.show()

### ⚡ Validación cruzada

Usemos validación cruzada para evaluar el entrenamiento de nuestro modelo, [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score).

La validación cruzada es una técnica de validación de nuestro entrenamiento. La usamos cuando queremos evaluar el desempeño del modelo usando sólo el conjunto de entrenamiento.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_train, y_train, cv=5)

In [ ]:
print(scores)
print(np.mean(scores))

In [ ]:
print(f"Accuracy en el conjunto de entrenamiento: {clf.score(X_test,y_test)}")
print(f"Accuracy CV en el conjunto de entrenamiento: {np.mean(scores)}")
print(f"Accuracy en el conjunto de prueba: {accuracy_score(y_test,y_pred)}")

### ⚡ ROC-AUC


In [ ]:
y_pred_probs = clf.predict_proba(X_test)
y_pred_probs.shape

Veamos las probabilidades predichas y la etiqueta predicha

In [ ]:
print(y_pred_probs[0])
print(y_pred[0])

Como son probabilidades, la suma de las componentes es 1

In [ ]:
np.sum(y_pred_probs[0])

Cuando se trata de un problema multiclase podemos escoger el enfoque `ovr` (one-vs-rest) o `ovo` (one-vs-one). En este caso, no se puede graficar *directamente* la curva ROC.

In [ ]:
from sklearn.metrics import roc_auc_score

score = roc_auc_score(y_test, y_pred_probs,multi_class='ovr')
print(f"ROC-AUC score: {score}")

## ⭕ Práctica *final* de clasificación

Vamos a retomar el dataset de la sesión pasada (dataset PIMA). El objetivo es tener el mejor modelo posible con cada uno de los siguientes algoritmos:

* SVM
* Decision Tree
* Random Forest
* Regresión Logística

Con esto haremos una comparación entre ellos.

Los pasos a seguir son:

1. Prepara el dataset para los algoritmo, recuerda que hay algunos valores faltantes. Además, recuerda reescalar los datos apropiadamente.

2. Usando como dataset el dataset preprocesado del paso anterior, realiza una busqueda de parámetros con cada algoritmo de acuerdo a las siguientes opciones:

* SVM
 * C: 0.1,1,10,100
 * kernel: lineal, polinomial, rbf
 * grados (polinomial): 2,3,5
* Decision Tree
 * criterion: gini, entropy, log_loss
 * max_depth: None, 10, 20,
 * min_samples_split: 2, 3, 5
* Random Forest
 * criterion: gini, entropy, log_loss
 * max_depth: None, 10, 20,
 * min_samples_split: 2, 3, 5
* Regresión Logística
 * C: 0.1,1,10
 * penalty: l1, l2, elasticnet, None

3. Considerando los 4 mejores modelos anteriores. ¿Qué clasificador tiene mejor rendimiento en este dataset? Para esto toma en cuenta el accuracy en el conjunto de prueba.

In [ ]:
import pandas as pd

url = 'https://github.com/DCDPUAEM/DCDP/raw/main/03%20Machine%20Learning/data/diabetes.csv'
df = pd.read_csv(url,index_col=0)
df

In [ ]:
'''
---------------
    TU CÓDIGO
--------------
'''

Hay más clasificadores que es importante revisar. Con las herramientas que ya cuentas, ya puedes revisarlos por tu cuenta:

* [K-nearest neighbors classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)
* [Quadratic Discriminant Analysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html#sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis)
* [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier)
* [Gaussian Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes)
* ...

[Más información](https://scikit-learn.org/stable/supervised_learning.html), [comparación](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).